In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import keras_tuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [6]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

    return model

In [7]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='intro_to_kt'
)

2022-03-24 19:59:05.260288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-24 19:59:05.284587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-24 19:59:05.284935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-24 19:59:05.285774: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [8]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [9]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 34s]
val_accuracy: 0.8619999885559082

Best val_accuracy So Far: 0.8914166688919067
Total elapsed time: 00h 08m 04s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 384 and the optimal learning rate for the optimizer
is 0.001.



In [10]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4963 - accuracy: 0.8242 - val_loss: 0.4342 - val_accuracy: 0.8427
Epoch 2/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3699 - accuracy: 0.8654 - val_loss: 0.3903 - val_accuracy: 0.8568
Epoch 3/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3340 - accuracy: 0.8757 - val_loss: 0.3434 - val_accuracy: 0.8753
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3067 - accuracy: 0.8857 - val_loss: 0.3315 - val_accuracy: 0.8801
Epoch 5/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2890 - accuracy: 0.8927 - val_loss: 0.3185 - val_accuracy: 0.8846
Epoch 6/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2701 - accuracy: 0.8994 - val_loss: 0.3380 - val_accuracy: 0.8809
Epoch 7/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2580 - accuracy: 0.9045 - val_loss: 0.3135 - val_accuracy:

In [11]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/40
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4985 - accuracy: 0.8227 - val_loss: 0.3961 - val_accuracy: 0.8604
Epoch 2/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3696 - accuracy: 0.8660 - val_loss: 0.3726 - val_accuracy: 0.8654
Epoch 3/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3319 - accuracy: 0.8771 - val_loss: 0.3630 - val_accuracy: 0.8683
Epoch 4/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3084 - accuracy: 0.8865 - val_loss: 0.3348 - val_accuracy: 0.8799
Epoch 5/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2863 - accuracy: 0.8932 - val_loss: 0.3185 - val_accuracy: 0.8852
Epoch 6/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2742 - accuracy: 0.8974 - val_loss: 0.3498 - val_accuracy: 0.8764
Epoch 7/40
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2582 - accuracy: 0.9042 - val_loss: 0.3361 - val_accuracy:

In [12]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5125 - accuracy: 0.8854
[test loss, test accuracy]: [0.5124523639678955, 0.8853999972343445]
